# More Users

Ok, the main point of this chapter is to go from our state of assuming that all users will share a common to-do list on the site to the state where we support multiple independent users.  The injunction with which we begin is, as always in this book, to consider first how we would confirm that whatever implementation we choose to realize this goal *is* in fact successful.

Here's an interesting point: he suggests that our functional tests file represents the closest thing that we have for a design document for this project.  I like that way of looking at the TDD paradigm: rather than keeping everything scattered and nebulous in your mind, you could be using the files storing your functional tests as a way to plan around the project; what should go where.

In [2]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import time
import unittest

MAX_WAIT = 10

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def wait_for_row_in_list_table(self, row_text):
        start_time = time.time()
        while True:
            try:
                table = self.browser.find_element_by_id('id_list_table')
                rows = table.find_elements_by_tag_name('tr')
                self.assertIn(row_text, [row.text for row in rows])
                return
            except (AssertionError, WebDriverException) as e:
                if time.time() - start_time > MAX_WAIT:
                    raise e
                time.sleep(0.5)

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)

        # The page updates again, and now shows both items on her list
        self.wait_for_row_in_list_table('1: Buy peacock feathers')
        self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Edith wonders whether the site will remember her list.  Then she sees
        # that the site has generated a unique URL for her -- there is some 
        # explanatory text to that effect.
        self.fail('Finish the test!')

        # She visits that URL - her to-do list is still there.

        # Satisfied, she goes back to sleep

Overwriting functional_tests/tests.py


# Agile vs. "Big Design Up Front"

Ok, another interesting point about design philosophy follows immediately.  Harry says that the traditional way to tackle large engineering projects would be to try to plan everything out with paper (or at least schematic software), figuring out where everything should go before you start writing code.  TDD, on the other hand, was derived from or otherwise aligned with the [AGILE design movement](https://en.wikipedia.org/wiki/Agile_software_development), which emphasizes incremental, iterative design, rather than figuring you can make an overarching plan from the very get-go that will prove very robust to complications encountered when coding up the details.  Basically, get your hands dirty early on, and expect the need to refactor.

Of course, the advocated answer is still not really to just dive in mindlessly, rather, it's to think in mid-sized, modular chunks.  The emphasis is on delivering a "minimal viable product", getting it in the field where you can collect real-world feedback about its reception.  Furthermore, this mindset is meant to counter the temptation to add on additional features and functionality that you might envision in a "perfect" or otherwise optimal app.  The initialism is "**YAGNI**", for "you ain't gonna need it": let the users tell you if something is lacking.

In this case, the plan to support multiple users involves having the server return customized URLs for each user, so they see different content based on the recognition of their particular credentials, and thus previously stored list items.


## REST

There's another term here, the REST (Representational State Transfer) API paradigm, which sounds like its gist is that the URL returned for any part of a web app/site should reflect the "data structure" of the content.  In practice, Harry suggests that the trailing part of the URL for a to-do list should contain:

* `lists/new` for a page that accepts POST requests to generate new lists
* `lists/list_name` to view the contents of an existing list
* `lists/list_name/add_item` for a page that accepts POST requests to add to a list

## Next Steps

He suggests that refactoring to support multiple users will involve creating an entirely new functional test; not sure yet if that means a new file, a new class in our existing module, or just new methods within the existing class.  ...Ok, it is quickly revealed that it's the latter option: new methods tacked on to our existing setup.  Ah, ok, but also some refactoring of existing ones; we're not just going to leave in place the soon-to-be-irrelevant methods that assumed that all visitors to the site shared one list.  So some shuffling around of code within the tests.

In [4]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import time
import unittest

MAX_WAIT = 10

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def wait_for_row_in_list_table(self, row_text):
        start_time = time.time()
        while True:
            try:
                table = self.browser.find_element_by_id('id_list_table')
                rows = table.find_elements_by_tag_name('tr')
                self.assertIn(row_text, [row.text for row in rows])
                return
            except (AssertionError, WebDriverException) as e:
                if time.time() - start_time > MAX_WAIT:
                    raise e
                time.sleep(0.5)

    def test_can_start_a_list_for_one_user(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)

        # The page updates again, and now shows both items on her list
        self.wait_for_row_in_list_table('1: Buy peacock feathers')
        self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Satisfied, she goes back to sleep

    def test_multiple_users_can_start_lists_at_different_urls(self):
        # Edith starts a new to-do list
        self.browser.get(self.live_server_url)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Buy peacock feathers')
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # She notices that her list has a unique URL
        edith_list_url = self.browser.current_url
        self.assertRegex(edith_list_url, '/lists/.+')

        # Now a new user, Francis, comes along to the site.

        ## We use a new browser session to make sure that no information
        ## of Edith's is coming through from cookies, etc.
        self.browser.quit()
        self.browser = webdriver.Firefox()

        # Francis visits the home page.  There is no sign of Edith's list
        self.browser.get(self.live_server_url)
        page_text = self.browser.find_element_by_tag_name('body').text
        self.assertNotIn('Buy peacock feathers', page_text)
        self.assertNotIn('make a fly', page_text)

        # Francis starts a new list by entering a new item.
        # He is less intersting than Edith...
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Buy milk')
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy milk')

        # Francis gets his own unique URL
        francis_list_url = self.browser.current_url
        self.assertRegex(francis_list_url, '/lists.+')
        self.assertNotEqual(francis_list_url, edith_list_url)

        # Again, there is no trace of Edith's list
        page_text = self.browser.find_element_by_tag_name('body').text
        self.assertNotIn('Buy peacock feathers', page_text)
        self.assertIn('Buy milk', page_text)

        # Satisfied, they both go back to sleep

Overwriting functional_tests/tests.py


Harry explains that the double-hash-mark lines represent *meta-comments*, which is really a bit of an aside to the dev reviewing the test, explaining *why* a particular step is being coded into the test that the user themselves wouldn't necessarily execute.  It's breaking the flow of assuming that the comments and code all represent a user's experience in navigating the site.  Here, it's assumed that another dev wouldn't understand why we're closing the browser (because it's not yet clear that a separate user is about to come on).

Now we run the modified functional test, hoping that `test_can_start_a_list_for_one_user` passes and `test_multiple_users_can_start_lists_at_different_urls` will fail...

In [1]:
!python manage.py test functional_tests

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.F
FAIL: test_multiple_users_can_start_lists_at_different_urls (functional_tests.tests.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\functional_tests\tests.py", line 80, in test_multiple_users_can_start_lists_at_different_urls
    self.assertRegex(edith_list_url, '/lists/.+')
AssertionError: Regex didn't match: '/lists/.+' not found in 'http://localhost:64258/'

----------------------------------------------------------------------
Ran 2 tests in 15.636s

FAILED (failures=1)


And that *is* the outcome anticipated by the text.  He suggests a commit to the remote repo at this point.